In [ ]:
from skimage import img_as_float, img_as_uint
from skimage import io 
from glob import glob
from skimage import transform, filters
import os
from lsm_utils import process_pmm_datasets, screen_background
from pycromanager import Dataset as PDataset
from os import path
import random
from skimage.filters import threshold_otsu, threshold_triangle, threshold_yen
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


In [ ]:
process_pmm_datasets('pmm_datasets/PA2081b_part2_clean', 'datasets/PA_2081b_part2/target')

In [ ]:
process_pmm_datasets('pmm_datasets/PA2081b_part2_noisy', 'datasets/PA_2081b_part2/input')

In [ ]:
fnames = glob('datasets/PA_2081b_part2/target/*.tif') + glob('datasets/PA_2081b_part2/input/*.tif')

In [ ]:
sampled = random.sample(fnames, min(5000, len(fnames)))
intensities = []
for fname in sampled:
    arr = io.imread(fname)
    mask = arr > (threshold_otsu(arr))
    arr = arr * mask
    intensity = np.sum(arr) / np.count_nonzero(mask)
    intensities.append(intensity)
threshold = (np.min(np.array(intensities))*0.25 + np.mean(np.array(intensities))*0.75)

In [ ]:
sampled = random.sample(fnames, min(5000, len(fnames)))
intensities = []
for fname in sampled:
    arr = io.imread(fname)
    mask = arr > (threshold_otsu(arr))
    arr = arr * mask
    intensity = np.sum(arr) / np.count_nonzero(mask)
    intensities.append(intensity)
threshold = (np.min(np.array(intensities))*0.25 + np.mean(np.array(intensities))*0.75)
remove_list = []
for fname in tqdm(fnames):
    arr = io.imread(fname)
    mask = arr > (threshold_otsu(arr))
    arr = arr * mask
    intensity = np.sum(arr) / np.count_nonzero(mask)
    if intensity < threshold:
        if np.random.uniform(0, 1)>0.1:
            remove_list.append(fname)

In [ ]:
remove_name_list = list(set([os.path.basename(i) for i in remove_list]))

In [ ]:
file_dirs = [i for i in fnames if os.path.basename(i) not in remove_name_list]

In [ ]:
_ = plt.hist(intensities, bins=100)
plt.show() 

In [ ]:
(np.min(np.array(intensities))*0.25 + np.mean(np.array(intensities))*0.75)

In [ ]:
len([i for i in intensities if i<threshold])

In [ ]:
for fname in sampled:
    arr = io.imread(fname)
    mask = arr > (threshold_otsu(arr))
    arr = arr * mask
    intensity = np.sum(arr) / np.count_nonzero(mask)
    if intensity < 15000:
        plt.imshow(arr)
        plt.show()

In [ ]:
fnames = glob('datasets/PA_2081b_part2/target/*.tif') + glob('datasets/PA_2081b_part2/input/*.tif')
exclude_list = screen_background(fnames)

In [ ]:
remove_name_list = list(set([os.path.basename(i) for i in exclude_list]))
file_dirs = [i for i in fnames if os.path.basename(i) not in remove_name_list]

In [ ]:
len(fnames) - len(file_dirs)

In [ ]:
len(remove_name_list)*2

In [ ]:
fnames_t = glob('datasets/PA_2081b_part2/target/*.tif')
file_dirs_t = [i for i in fnames_t if os.path.basename(i) not in remove_name_list]

In [ ]:
fnames_i = glob('datasets/PA_2081b_part2/input/*.tif')
file_dirs_i = [i for i in fnames_i if os.path.basename(i) not in remove_name_list]

In [2]:
from skimage import filters, morphology, io, img_as_uint, exposure, transform
from skimage.restoration import denoise_tv_chambolle, denoise_wavelet
from glob import glob
from tqdm import tqdm
import shutil

In [57]:
fnames = glob('datasets/PA_2081b/input/*.tif')
intensity = []
for fname in tqdm(fnames):
    intensity.append(np.mean(io.imread(fname)))
    
thresh = np.percentile(np.array(intensity), 90)
for fname in tqdm(fnames):
    if np.mean(io.imread(fname)) > thresh:
        shutil.copyfile(fname, fname.replace('input', 'evaluation/input'))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 72360/72360 [00:29<00:00, 2419.83it/s]


In [70]:
fnames = glob('datasets/PA_2081b/evaluation/input/*.tif')
for fname in tqdm(fnames):
    shutil.copyfile(fname.replace('evaluation/input', 'target'), fname.replace('input', 'target'))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 7236/7236 [00:03<00:00, 2266.95it/s]


In [7]:
fnames = glob('datasets/PA_2081b/evaluation/input/*.tif')
for fname in tqdm(fnames):
    img_arr = io.imread(fname)
    
    # img_mf = filters.median(img_arr, footprint=morphology.square(3))
    img_mf = denoise_wavelet(img_arr, rescale_sigma=True)
    # img_mf = denoise_tv_chambolle(img_arr, weight=0.05)
    # img_mf = img_arr
    
    img_mf = transform.rescale(img_mf, 2, order=3)
    # io.imsave(fname, img_as_uint(img_mf), check_contrast=False)

  7%|███████▉                                                                                                   | 539/7236 [00:22<04:33, 24.50it/s]


KeyboardInterrupt: 

In [71]:
fnames = glob('datasets/PA_2081b/evaluation/target/*.tif')
for fname in tqdm(fnames):
    img_arr = img_as_uint( io.imread(fname) )
    img_mf = exposure.rescale_intensity(img_arr, in_range=(6265, 37193), out_range=(0, 1))
    io.imsave(fname, img_as_uint(img_mf), check_contrast=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7236/7236 [00:46<00:00, 154.60it/s]


In [63]:
import yaml
config = yaml.load(open("model_config.yaml", "r"), Loader=yaml.FullLoader)

In [65]:
config

{'adv-interval': 1,
 'average-factor': 50,
 'batch-size': 8,
 'blindspot-rate': 0.05,
 'cnn-base-channel': 8,
 'dataset': 'datasets/PA_2081b',
 'gan-lambda': 10,
 'gpu': True,
 'image-channel': 1,
 'image-extension': 'tif',
 'iter-per-epoch': 500,
 'iterations': 100000,
 'learning-rate': 0.0001,
 'load-weights': 'supervised2022-07-22_21-45-35',
 'loss-gain': 1000,
 'norm-range': [5432, 43954],
 'norm-range-target': [6265, 37193],
 'only-pixel-loss': False,
 'percep-lambda': 0.5,
 'resolution': 512,
 'test-interval': 5,
 'threads': 32,
 'up-scale-factor': 2}